In [211]:
# % pip install timezonefinder

In [212]:
import os
import operator
import numpy as np
import astropy
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body
from datetime import datetime
from timezonefinder import TimezoneFinder
import pytz

In [213]:
def get_nouns(filename):
    nouns = []
    with open(filename, 'r') as f:
        for line in f:
            nouns.append(line.strip())
    return nouns

In [214]:
def get_object_icrs(time, location, object):
    SolarSystemBodies = ['sun', 'mercury', 'venus', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto']
    # Convert to lower case
    object = object.lower()
    if object in SolarSystemBodies:
        # Get the RA and Dec of the object if in solar system
        with solar_system_ephemeris.set('builtin'): # Use for solar system bodies
            RADe_object = get_body(object, time, location) 
            # Get ICRS coordinates
            RADe_object = RADe_object.icrs
    else:
        # A celestial object in ICRS outside the solar system
        RADe_object = astropy.coordinates.get_icrs_coordinates(object)
    return RADe_object

In [215]:
# Get location at specific observation site
observe_location = EarthLocation.of_site('greenwich') 
# EarthLocation.of_address('New York, NY') if use arbitrary addresses
# EarthLocation.get_site_names()  if need to see list of sites
tz_name = TimezoneFinder().timezone_at(lng=observe_location.lon.degree,
                                       lat=observe_location.lat.degree)
tz = pytz.timezone(tz_name)
now = datetime.now(tz) # Get time at observatory
time = Time(now) # Observation time. Convert to Astropy format

In [217]:
object = 'M31' # get_nouns(exact_object.txt) -> This needs work
download_type = 'gif' # Or 'fits'
data_source = "https://archive.stsci.edu/cgi-bin/dss_form"

In [218]:
RADe_object = get_object_icrs(time, observe_location, object) # Or 'sun' if looking in solar system but sky survey cant retrieve solar system image
RA = RADe_object.ra.deg
DE = RADe_object.dec.deg
print('RA = ', RA)
print('DE = ', DE)

RA =  10.6847083
DE =  41.26875


In [219]:
#%pip install selenium # Install Selenium for feeding the web browser
#%pip install webdriver-manager # Install webdriver-manager for managing the web driver

In [220]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [221]:
# Automating locate elements for data retrieving
browser.get(data_source)
ra = browser.find_element(By.NAME, 'r')
de = browser.find_element(By.NAME, 'd')
file = browser.find_element(By.XPATH, value="//select[@name='f']")
terms = browser.find_element(By.XPATH, value="//input[@value='      RETRIEVE IMAGE      ']")

In [222]:
# Feed RA-DE to the website
ra.send_keys(RA)
de.send_keys(DE)
file.send_keys(download_type) 
terms.click()

In [223]:
#closing the browser
#browser.quit()